# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [4]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [7]:
df_rates.userID.values

array([   75,    75,    75, ..., 71534, 71534, 71534])

In [8]:
df_rates.movieID.values

array([    3,    32,   110, ..., 48516, 61075, 62049])

In [9]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [10]:
df_rates

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30
...,...,...,...,...,...,...,...,...,...
855593,71534,44555,4.0,3,12,2007,3,5,38
855594,71534,46578,4.0,3,12,2007,2,56,44
855595,71534,48516,4.5,3,12,2007,2,53,46
855596,71534,61075,5.0,10,10,2008,9,56,5


In [11]:
df_rates.userID.values

array([   75,    75,    75, ..., 71534, 71534, 71534])

In [12]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [13]:
idx

0        True
1        True
2        True
3        True
4        True
         ... 
10192    True
10193    True
10194    True
10195    True
10196    True
Name: id, Length: 10197, dtype: bool

In [14]:
df_movies

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10192,65088,Bedtime Stories,960731,Más allá de los sueños,http://ia.media-imdb.com/images/M/MV5BMjA5Njk5...,2008,bedtime_stories,4.4,104,26,...,25,4.7,26,6,20,23,3.5,108877,63,http://content6.flixster.com/movie/10/94/33/10...
10193,65091,Manhattan Melodrama,25464,El enemigo público número 1,http://ia.media-imdb.com/images/M/MV5BMTUyODE3...,1934,manhattan_melodrama,7,12,10,...,83,0,4,2,2,50,3.7,344,71,http://content9.flixster.com/movie/66/44/64/66...
10194,65126,Choke,1024715,Choke,http://ia.media-imdb.com/images/M/MV5BMTMxMDI4...,2008,choke,5.6,135,73,...,54,4.9,26,8,18,30,3.3,13893,55,http://content6.flixster.com/movie/10/85/09/10...
10195,65130,Revolutionary Road,959337,Revolutionary Road,http://ia.media-imdb.com/images/M/MV5BMTI2MzY2...,2008,revolutionary_road,6.7,194,133,...,68,6.9,36,25,11,69,3.5,46044,70,http://content8.flixster.com/movie/10/88/40/10...


In [15]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

## Матрица рейтингов

In [16]:
from scipy.sparse import coo_matrix, csr_matrix

In [17]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [18]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in COOrdinate format>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [19]:
R = R.tocsr()

In [20]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in Compressed Sparse Row format>

Теперь, например, рейтинги для первого пользователя можно достать так:

In [21]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [22]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [23]:
user_1_rated = (user_1 != 0)
user_1_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 55 stored elements in Compressed Sparse Row format>

Можем их "индексировать"

In [24]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [25]:
user_1.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [26]:
(user_1).multiply(user_2)

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

и скалярно

In [27]:
user_1.dot(user_2.T)

<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [28]:
user_1.dot(user_2.T)[0, 0]

216.75

И превращать разреженную матрицу (вектор) в плотную

In [29]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


### _Решение_

In [30]:
def cosine_similarity_pair_users(u, v):
    rated_u = (u != 0)
    rated_v = (v != 0)
    common_items = (rated_u).multiply(rated_v)
    if common_items.nnz > 2:
        up = u.dot(v.T)[0, 0]
        down1 = u.dot(u.T)[0, 0]
        down2 = v.dot(v.T)[0, 0]
        down1 = down1 ** 0.5
        down2 = down2 ** 0.5
        down = down1 * down2
        s = up / down
        return s
    else:
        return 0

In [31]:
round(cosine_similarity_pair_users(R[146], R[239]), 8)

0.17772036

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

print(cosine_similarity(R[146], R[239]))


[[0.17772036]]


In [33]:
def cosine_similarity_pair_users(u, v):
    u_rated = (u != 0)
    v_rated = (v != 0)
    common_items = (u_rated).multiply(v_rated)
    if common_items.nnz > 2:
        up = u.dot(v.T)[0, 0]
        down = np.linalg.norm(u[common_items]) * np.linalg.norm(v[common_items])
        return up / down
    else:
        return 0.0

In [34]:
cosine_similarity_pair_users(R[146], R[239])

0.9228878934866824

In [35]:
answer1 = round(cosine_similarity_pair_users(R[146], R[239]),3)
print(answer1)

0.923


Введите значение answer1 на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy

## Функция нахождения пользователей, схожих с данным. 

Реализуйте функцию <font color = "blue">similar_users</font>(u, R, n_neigbours) которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей 
и возвращает отсортированный массив пользователей (сортировка по неубыванию), максимально похожих на данного. Для сортировки используйте np.argsort без параметров. (https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html) (Сам пользователь будет в этом списке на первом месте). Эту функцию вы сможете использовать далее. 

In [36]:
def similar_users(u, R, n_neigbours):
    return [cosine_similarity_pair_users(R[u], R[x])
                  for x in range(R.shape[0])]

In [37]:
g = similar_users(42, R, 10)
g

[0.9763098382685473,
 0.9812618624864466,
 0.9936145992563616,
 0.9605113957257359,
 0.9628831495544733,
 0.9166904540041484,
 0.9625052521193127,
 0.95625926439942,
 0.9431394093994782,
 0.978724470911071,
 0.9801051972750358,
 0.9756806575746247,
 0.9816555652542659,
 0.9724709325699408,
 0.9727200749774173,
 0.9861993375448909,
 0.8837296772722445,
 0.9797841073628432,
 0.9601217501861145,
 0.9757587185624222,
 0.9658541321978608,
 0.9558780972669719,
 0.9365737677561641,
 0.9752856867523659,
 0.9601269121187832,
 0.9613962936367306,
 0.9685640873778264,
 0.9739732885237145,
 0.9524808261483962,
 0.9808552017709151,
 0.9644366093719039,
 0.9780770267951381,
 0.9734932197745109,
 0.9754757433555121,
 0.9820310254975614,
 0.9709389483261317,
 0.9773990374278256,
 0.9702673465728662,
 0.9727111038706799,
 0.9420553381257555,
 0.9563612943701708,
 0.9658124326306499,
 1.0000000000000002,
 0.9543173311186227,
 0.9778858220606851,
 0.9819154090293745,
 0.9170961412186058,
 0.9645233661595

In [38]:
g.sort()
g[:-10:-1]

[1.0000000000000002,
 0.9968640410822807,
 0.9960192126181785,
 0.9959100033104787,
 0.9958705948858224,
 0.9936145992563616,
 0.9933809269777035,
 0.9932974129346572,
 0.9929666136638231]

In [39]:
def similar_users(u, R, n_neigbours):
    return np.argsort([cosine_similarity_pair_users(R[u], R[x])
                  for x in range(R.shape[0])])[:-n_neigbours:-1]

In [40]:
s=similar_users(42, R, 10)
s

array([  42,  281,  633,  724,  815,    2,  620,  650, 1692])

In [41]:
answer2 = np.array2string(similar_users(42, R, 10)).replace(' ','').replace('[','').replace(']','')

In [42]:
len(answer2)

25

In [43]:
answer2

'4228163372481526206501692'

Введите значение answer2 без кавычек  на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy. Это будет строка из 29 символов, которая начинается на 42.

## Функция прогнозирования рейтинга

Реализуйте функцию <font color = "blue">rate_items_user</font>(u, R, n_neigbours), которая принимает на входе:
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей <font color = "red">(Теперь обратите внимание, несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)</font>

и возвращает вектор с предсказанными рейтингами по всем фильмам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [44]:
def rate_items_user(u, R, n_neigbours):
    predictions = csr_matrix((1, R.shape[1]))
    s_uv = np.array([cosine_similarity_pair_users(R[u], R[i]) for i in range(R.shape[0])]) # все косинусные меры до ближайщих соседей
    similar_users = np.argsort(s_uv)[::-1]
    znam = 0
    for v in similar_users[: n_neigbours]:
        if v != u:
            predictions += s_uv[v] * R[v]
            znam += abs(s_uv[v])
    return (predictions / znam)

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).

* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

Т.е. предсказанные рейтинги можно получить так:
R_hat = <font color = "blue">rate_items_user</font>(20, R, n_neigbours=30). При сортировке фильмов по рейтингу используйте функцию <font color = "blue">argsort </font> без параметров.

### _Решение_

In [45]:
rated_items = (R[20] == 0)
rated_items

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 10076 stored elements in Compressed Sparse Row format>

In [46]:
R_hat = rate_items_user(20, R, n_neigbours=30)

In [47]:
unseen_ratings = R_hat.multiply(rated_items)
unseen_ratings

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 1437 stored elements in Compressed Sparse Row format>

In [48]:
unseen_ratings = unseen_ratings.toarray()[0]
unseen_ratings

array([1.08599581, 0.58590957, 0.10334154, ..., 0.        , 0.        ,
       0.        ])

In [49]:
idxTop = unseen_ratings.argsort()[::-1]

In [50]:
unseen_ratings[idxTop]

array([2.5174164 , 2.24147475, 2.15522858, ..., 0.        , 0.        ,
       0.        ])

In [51]:
top5 = idxTop[:5]

## Генерация ответа

In [52]:
answer3 = ', '.join(str(i) for i in top5)

In [53]:
answer3

'2614, 306, 343, 5573, 6720'

Полученную строку введите на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy Формат ответа - строка вида "X, X, X, X, X", где X - идентификаторы. Вводить ответ следует без кавычек.